In [29]:
import cv2
from sklearn import svm
from scipy.cluster.vq import kmeans, vq
import numpy as np
import pandas as pd
import glob
from skimage.filters import threshold_yen
from skimage.exposure import rescale_intensity
from scipy.stats import skew
from itertools import chain
from skimage.feature import greycomatrix, greycoprops
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler as sc


In [30]:
Apple_training_data='dataset\\Apples\\Train\\*.jpg'
Orange_training_data='dataset\\Oranges\\Train\\*.jpg'
Mango_training_data='dataset\\Mangoes\\Train\\*.jpg'
Test_data='dataset\\test\\*.jpg'

Apple_hue_arr = []   #Info of hue of  each apple pic
Orange_hue_arr = []  #Info of hue of  each orange pic
Mango_hue_arr = []   #Info of hue of  each mango pic
Apple_des_list = []
Orange_des_list = []
Mango_des_list = []
sift = cv2.SIFT_create()


In [31]:
def get_all_glcm_features(gray_scale_img):
    """
    Given a grayscale image with graylevels from 0 - 255, this function returns the contrast
    and the homogeneity features of the image with the help of GLCM
    """
    # Tip: Make sure you understand the input-output of everything you write, 
    # not doing that results in bugs that make you believe the lab is long
    
    #size of co-occ matrix = number of gray levels
    image_array = np.array(gray_scale_img)
    #print('first pixel= ', image_array[0][0])
    coocurrence_matrix = greycomatrix(image_array, [1], [0])
    contrast = greycoprops(coocurrence_matrix, 'contrast')
    homogeneity = greycoprops(coocurrence_matrix, 'homogeneity')
    #mean = greycoprops(coocurrence_matrix, 'mean')
    energy = greycoprops(coocurrence_matrix, 'energy')
    #entropy = greycoprops(coocurrence_matrix, 'entropy')
    #variance = greycoprops(coocurrence_matrix, 'variance')
    correlation = greycoprops(coocurrence_matrix, 'correlation')
    return contrast, homogeneity, energy, correlation

In [32]:
for file in glob.glob(Apple_training_data):    
    img = cv2.imread(file)  #read apple images
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  #convert from RBG to HSV
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contrast, homogeneity, energy, correlation= get_all_glcm_features((img_gray * 255).astype(np.uint8))
    kpts, des = sift.detectAndCompute(img_gray,None)
    if len(kpts) < 1:
        no_kpts = np.zeros((1, sift.descriptorSize()), np.float32)
        Apple_des_list.append((file, no_kpts))
    else:
        Apple_des_list.append((file, des)) 
    h,s,v=cv2.split(img)  #Split the the channels of each image
    hue_mean=np.mean(h)  #calculate the mean of hue channel of each image
    hue_var=np.var(h)    #calculate the var of hue channel of each image
    flatten_h = list(np.concatenate(h).flat)  #convert the 2-D image to 1-D image
    flatten_s = list(np.concatenate(s).flat)
    hue_skewness=skew(flatten_h)   #calculate the skewness of hue channel of each image
    image_hue_info=[]
    image_hue_info.append(hue_mean)
    image_hue_info.append(hue_var)
    image_hue_info.append(hue_skewness)
    #######################################################################
    image_hue_info.append(contrast[0][0])
    image_hue_info.append(homogeneity[0][0])
    image_hue_info.append(energy[0][0])
    image_hue_info.append(correlation[0][0])
    #########################################################################
    Apple_hue_arr.append(image_hue_info)
print(len(Apple_hue_arr))

72


In [33]:
# Stack all the descriptors vertically in a numpy array
descriptors = Apple_des_list[0][1]
for file, descriptor in Apple_des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor))

In [34]:
#kmeans works only on float, so convert integers to float
descriptors_float = descriptors.astype(float)

k = 60  #k means awal 60 clusters
voc, variance = kmeans(descriptors_float, k, 1)

In [35]:
Apple_features = np.zeros((72, k), "float32")
for i in range(72):
    words, distance = vq(Apple_des_list[i][1],voc)
    for w in words:
        Apple_features[i][w] += 1

In [36]:
Apples_all_features=np.concatenate((Apple_hue_arr,Apple_features.tolist()),axis=1).tolist()

In [37]:
for file in glob.glob(Orange_training_data):    
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contrast, homogeneity, energy, correlation= get_all_glcm_features((img_gray * 255).astype(np.uint8))
    kpts, des = sift.detectAndCompute(img_gray,None)
    if len(kpts) < 1:
        no_kpts = np.zeros((1, sift.descriptorSize()), np.float32)
        Orange_des_list.append((file, no_kpts))
    else:
        Orange_des_list.append((file, des)) 
    h,s,v=cv2.split(img)
    hue_mean=np.mean(h)  #calculate the mean of hue channel of each image
    hue_var=np.var(h) 
    flatten_h = list(np.concatenate(h).flat)
    hue_skewness=skew(flatten_h)
    image_hue_info=[]
    image_hue_info.append(hue_mean)
    image_hue_info.append(hue_var)
    image_hue_info.append(hue_skewness)
    #######################################################################
    image_hue_info.append(contrast[0][0])
    image_hue_info.append(homogeneity[0][0])
    image_hue_info.append(energy[0][0])
    image_hue_info.append(correlation[0][0])
    #########################################################################
    Orange_hue_arr.append(image_hue_info) 

In [38]:
# Stack all the descriptors vertically in a numpy array
descriptors = Orange_des_list[0][1]
for file, descriptor in Orange_des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor))

In [39]:
#kmeans works only on float, so convert integers to float
descriptors_float = descriptors.astype(float)

k = 60  #k means awal 60 clusters
voc, variance = kmeans(descriptors_float, k, 1)

In [40]:
Orange_features = np.zeros((70, k), "float32")
for i in range(70):
    words, distance = vq(Orange_des_list[i][1],voc)
    for w in words:
        Orange_features[i][w] += 1

In [41]:
Oranges_all_features=np.concatenate((Orange_hue_arr,Orange_features.tolist()),axis=1).tolist()

In [42]:
for file in glob.glob(Mango_training_data):    
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contrast, homogeneity, energy, correlation= get_all_glcm_features((img_gray * 255).astype(np.uint8))
    kpts, des = sift.detectAndCompute(img_gray,None)
    if len(kpts) < 1:
        no_kpts = np.zeros((1, sift.descriptorSize()), np.float32)
        Mango_des_list.append((file, no_kpts))
    else:
        Mango_des_list.append((file, des)) 
    h,s,v=cv2.split(img)
    hue_mean=np.mean(h)
    hue_var=np.var(h)
    flatten_h = list(np.concatenate(h).flat)
    hue_skewness=skew(flatten_h)
    image_hue_info=[]
    image_hue_info.append(hue_mean)
    image_hue_info.append(hue_var)
    image_hue_info.append(hue_skewness)
    #######################################################################
    image_hue_info.append(contrast[0][0])
    image_hue_info.append(homogeneity[0][0])
    image_hue_info.append(energy[0][0])
    image_hue_info.append(correlation[0][0])
    #########################################################################
    Mango_hue_arr.append(image_hue_info) 

In [43]:
# Stack all the descriptors vertically in a numpy array
descriptors = Mango_des_list[0][1]
for file, descriptor in Mango_des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor))

In [44]:
#kmeans works only on float, so convert integers to float
descriptors_float = descriptors.astype(float)

k = 60  #k means awal 60 clusters
voc, variance = kmeans(descriptors_float, k, 1)

In [45]:

Mango_features = np.zeros((78, k), "float32")
for i in range(78):
    words, distance = vq(Mango_des_list[i][1],voc)
    for w in words:
        Mango_features[i][w] += 1

In [46]:
Mangos_all_features=np.concatenate((Mango_hue_arr,Mango_features.tolist()),axis=1).tolist()

In [47]:
#print(len(Apple_hue_arr),len(Orange_hue_arr),len(Mango_hue_arr))
a=["Apple"]*len(Apples_all_features)
o=["Orange"]*len(Oranges_all_features)
m=["Mango"]*len(Mangos_all_features)
all_labels=a+o+m
print(len(all_labels))
print(all_labels)

all_features= Apples_all_features + Oranges_all_features + Mangos_all_features
print(len(all_features))
print(all_features[0])


220
['Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Apple', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange', 'Orange

#pca
pca = PCA()
x_scaled_pca = pca.fit_transform(all_features)

per_var = np.round(pca.explained_variance_ratio_*100,decimals=1)
labels =[str(x) for x in range(1, len(per_var)+1)]


plt.bar(x=range(1, len(per_var)+1), height=per_var)
plt.tick_params(
    axis='x',
    which='both',
    bottom= False,
    top= False,
    labelbottom=False)
plt.ylabel('percentage of explained variance')
plt.xlabel('principal Components')
plt.title('scree plot')
plt.show()

In [52]:
param_grid = [
    {
        'C': [0.5, 1, 10, 100],
        'gamma': ['scale',1,0.1,0.01,0.001,0.0001],
        'kernel': ['rbf','linear']
    },
]

optimal_params = GridSearchCV(
    svm.SVC(),
    param_grid,
    cv=20,
    scoring='accuracy',
    verbose=0
)

#print(len(all_features_scalled),len(y_train))
all_features_scaled = sc().fit_transform(all_features)

optimal_params.fit(all_features_scaled,all_labels)
print(optimal_params.best_params_)

{'C': 0.5, 'gamma': 'scale', 'kernel': 'rbf'}


In [53]:
clf_svm_rbf =  svm.SVC(random_state=0,C=0.5,gamma='scale',kernel='rbf')
clf_svm_rbf.fit(all_features_scaled,all_labels)

SVC(C=0.5, random_state=0)

all_features_scalled=scale(all_features)
clf_linear = svm.SVC(kernel='linear').fit(all_features_scalled[0:220,0:3].tolist(),all_labels)

In [62]:
Test_des_list=[]
Test_hue_arr=[]
for file in glob.glob(Test_data):    
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    contrast, homogeneity, energy, correlation= get_all_glcm_features((img_gray * 255).astype(np.uint8))
    kpts, des = sift.detectAndCompute(img_gray,None)
    if len(kpts) < 1:
        no_kpts = np.zeros((1, sift.descriptorSize()), np.float32)
        Test_des_list.append((file, no_kpts))
    else:
        Test_des_list.append((file, des)) 
    h,s,v=cv2.split(img)
    hue_mean=np.mean(h)  #calculate the mean of hue channel of each image
    hue_var=np.var(h) 
    flatten_h = list(np.concatenate(h).flat)
    hue_skewness=skew(flatten_h)
    image_hue_info=[]
    image_hue_info.append(hue_mean)
    image_hue_info.append(hue_var)
    image_hue_info.append(hue_skewness)
    #######################################################################
    image_hue_info.append(contrast[0][0])
    image_hue_info.append(homogeneity[0][0])
    image_hue_info.append(energy[0][0])
    image_hue_info.append(correlation[0][0])
    #########################################################################
    Test_hue_arr.append(image_hue_info) 



    # Stack all the descriptors vertically in a numpy array
descriptors = Test_des_list[0][1]
for file, descriptor in Test_des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor))


#kmeans works only on float, so convert integers to float
descriptors_float = descriptors.astype(float)

k = 60  #k means awal 60 clusters
voc, variance = kmeans(descriptors_float, k, 1)




Test_features = np.zeros((len(Test_hue_arr), k), "float32")
for i in range(len(Test_hue_arr)):
    words, distance = vq(Test_des_list[i][1],voc)
    for w in words:
        Test_features[i][w] += 1



Test_features_scaled = scale(Test_features.tolist())
print(Test_features[0],Test_features_scaled[0])

Test_hue_arr_scaled = scale(Test_hue_arr)
print(Test_hue_arr[0],Test_hue_arr_scaled[0])

Test_all_features=np.concatenate((Test_hue_arr_scaled,Test_features_scaled.tolist()),axis=1).tolist()

[ 81.  14.  55.  28.   7. 151.  24.  61.  54.  53.  12.  39.   9.  15.
  38.   1.  23.  37.  58.  39.  58.  62.  70.  20.  39.  73.  15.  26.
 170.  11.  29.  12.  28.  22.  42.  33.  28.   1.  47.  73.  23.  42.
  27.  18.  24.  11.  61.  12.  56.  52.  53.  21.  19.  61.  56.  51.
  36.  42.  25.  57.] [-0.25846954 -0.57475712 -0.06088423 -0.61688323 -0.70619813  0.55082274
 -0.67425702 -0.30658218 -0.02908573 -0.21904779 -0.65964596 -0.5391918
 -0.67957943 -0.6016993  -0.6627812  -0.63094987 -0.62601596 -0.50883097
 -0.19604642 -0.45338123 -0.41712416 -0.27677757  0.09316418 -0.58734673
 -0.47915511  0.06046792 -0.62154992 -0.58839075  0.93215911 -0.64358028
  0.23911544 -0.58359234 -0.55161507 -0.37657265 -0.53161841 -0.51090759
 -0.54083358 -0.59136666 -0.0976016  -0.35725953 -0.54125705 -0.34062461
  0.99665648 -0.62268594 -0.58781443 -0.58894897 -0.2968477  -0.60481304
 -0.20998927 -0.57477009 -0.72459579 -0.3546952  -0.47346692 -0.07139934
 -0.2665754   0.01949903 -0.44134485 -

In [63]:
#predict the fruit type in a new image
#print("LINEAR",clf_linear.predict(Test_all_features))
print("rbf",clf_svm_rbf.predict(Test_all_features))


rbf ['Orange' 'Orange' 'Orange' 'Mango' 'Mango' 'Mango' 'Mango' 'Apple'
 'Apple' 'Apple']
